In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import seaborn
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from pycaret.classification import *
from pycaret.classification import evaluate_model
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import selenium
from selenium import webdriver
import random
from selenium.webdriver.common.by import By
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
pd.set_option('display.max_columns', 100)

C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\llvmlite\llvmpy\__init__.py:4: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  "The module `llvmlite.llvmpy` is deprecated and will be removed in the "
C:\Users\idotz\anaconda3\envs\p37\lib\site-packages\llvmlite\llvmpy\core.py:9: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  "The module `llvmlite.llvmpy.core` is deprecated and will be removed in "
C:\Users\idotz\anaconda3\envs\p37\

In [2]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [52]:
setup_df = pd.read_csv('f_eng_df.csv', index_col = [0])
setup_df = setup_df[setup_df['odds_sum'] > 1.02]
setup_df.shape

(275, 97)

In [149]:
grid = setup(data=setup_df, target='target', session_id = 123,
            feature_ratio = True, feature_interaction = True)

,Description,Value
0,session_id,123
1,Target,target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(275, 97)"
5,Missing Values,True
6,Numeric Features,93
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [150]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7242,0.9061,0.6892,0.7372,0.7144,0.5615,0.5740,0.2030
xgboost,Extreme Gradient Boosting,0.7134,0.8788,0.6776,0.7239,0.7016,0.5452,0.5562,0.4150
gbc,Gradient Boosting Classifier,0.6926,0.8826,0.6492,0.6963,0.6750,0.5107,0.5277,1.2880
rf,Random Forest Classifier,0.6874,0.8485,0.6340,0.6677,0.6665,0.4971,0.5091,0.1850
et,Extra Trees Classifier,0.6818,0.8554,0.6279,0.6647,0.6623,0.4901,0.5024,0.1920
lda,Linear Discriminant Analysis,0.6676,0.7843,0.6464,0.6923,0.6693,0.4827,0.4907,0.0140
ada,Ada Boost Classifier,0.6403,0.7945,0.6162,0.6347,0.6186,0.4457,0.4576,0.1050
svm,SVM - Linear Kernel,0.5784,0.0000,0.5344,0.5758,0.5542,0.3342,0.3531,0.0310
knn,K Neighbors Classifier,0.5571,0.7286,0.5064,0.5503,0.5413,0.2900,0.2992,0.0580
nb,Naive Bayes,0.5532,0.8016,0.5769,0.7123,0.5640,0.3565,0.4001,0.0080


In [152]:
tuned = tune_model(best, target = 'recall')
print(tuned)

IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:43:49
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Fold,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


TypeError: __init__() got an unexpected keyword argument 'target'

In [145]:
training_data = get_config(variable="X_train")

In [146]:
training_data

,med_1,xg_on_target_(xgot)_diff,std_1,med_2,odds_sum,med_1_multiply_odds_sum,med_2_multiply_med_1
13,1.050000,2.11,0.008803,41.000000,1.053694,1.106379,43.049999
237,2.350000,-1.46,0.042295,3.200000,1.077318,2.531696,7.520000
66,1.363636,0.48,0.017721,10.000000,1.065892,1.453488,13.636364
88,1.700000,1.34,0.035365,7.500000,1.069395,1.817971,12.750000
271,10.750000,-0.46,0.832541,1.444444,1.071045,11.513737,15.527778
...,...,...,...,...,...,...,...
113,1.181818,1.60,0.019304,18.000000,1.068376,1.262626,21.272726
90,7.000000,-0.74,0.486554,1.500000,1.079794,7.558558,10.500000
19,1.292857,0.80,0.024728,13.000000,1.067795,1.380507,16.807144
240,3.200000,-0.27,0.100519,2.700000,1.082870,3.465185,8.640000


In [134]:
xgb = create_model('xgboost')
tuned = tune_model(xgb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7500,0.8865,0.6889,0.8015,0.7292,0.5885,0.6264
1,0.6500,0.7879,0.6074,0.6938,0.6589,0.4697,0.4825
2,0.6316,0.8070,0.5185,0.5441,0.5742,0.3667,0.3958
3,0.6316,0.8348,0.5648,0.6368,0.6313,0.4089,0.4108
4,0.4737,0.6555,0.4630,0.5138,0.4855,0.2017,0.2052
5,0.6316,0.8478,0.5463,0.6340,0.6211,0.4009,0.4087
6,0.7895,0.8348,0.7407,0.8182,0.7825,0.6514,0.6650
7,0.8421,0.9684,0.8296,0.8421,0.8421,0.7522,0.7522
8,0.7895,0.8964,0.7556,0.8194,0.7826,0.6681,0.6906


In [147]:
evaluate_model(tuned)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [148]:
predicted_df = predict_model(tuned, raw_score = True)
z = predicted_df
predicted_df = predicted_df[['med_0','med_1','med_2'] + list(predicted_df.columns[-5:])]
df = df.rename({'med_0':'0','med_1':'1','med_2':'2'}, axis = 1)
bo_po_dha = list(zip(predicted_df.columns[0:3],predicted_df.columns[-3:],['draw','home','away']))
for book_odds, pred_odds, dha in bo_po_dha:
    predicted_df[dha] = predicted_df.apply(lambda x: kelly(x[pred_odds], x[book_odds], 0.25), axis=1)
predicted_df.to_csv('pred_profits.csv')
df = predicted_df
dha = {'draw':'0','home':'1','away':'2'}
df = df.replace('no value', 0)
df[['draw','home','away']] = df[['draw','home','away']].astype(float)
for word, number in dha.items():
    df[f'{word}_profits'] = np.where(df['target'] == int(number),
                                    df[word] * (df['med_'+ number] - 1) * 100,
                                    -100*df[word])
df['profits'] = df[df.columns[-3:]].sum(axis = 1)
p = round(np.sum(df["profits"]), 3)
df = df.round(3)
df.to_csv(f'profits_{datetime.now():%Y%m%d}.csv')
print(f'number of bets made: {df.shape[0]}')
print(f'profit on $100 bankroll: ${p}')
print(df.sum()[-4:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.5783,0.8017,0.5465,0.6132,0.5726,0.3601,0.3741


KeyError: "['med_0'] not in index"

In [82]:
df.sort_values(by = 'home_profits')

,med_0,med_1,med_2,target,Label,Score_0,Score_1,Score_2,draw,home,away,draw_profits,home_profits,away_profits,profits
30,6.00,1.182,18.00,2,1,0.000,1.000,0.000,0.0,0.250,0.0,-0.0,-24.982,0.0,-24.982
16,4.90,1.250,15.00,2,1,0.000,1.000,0.000,0.0,0.250,0.0,-0.0,-24.975,0.0,-24.975
62,2.70,1.833,6.50,2,1,0.000,0.999,0.000,0.0,0.250,0.0,-0.0,-24.968,0.0,-24.968
20,4.45,1.316,11.00,0,1,0.000,1.000,0.000,0.0,0.250,0.0,0.0,-24.958,-0.0,-24.958
47,3.40,1.667,5.50,0,1,0.001,0.998,0.000,0.0,0.249,0.0,0.0,-24.893,-0.0,-24.893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,3.80,1.400,9.50,1,1,0.001,0.997,0.002,0.0,0.247,0.0,-0.0,9.898,-0.0,9.898
25,3.75,1.500,6.75,1,1,0.014,0.986,0.001,0.0,0.239,0.0,-0.0,11.968,-0.0,11.968
19,2.90,1.650,7.50,1,1,0.000,0.999,0.000,0.0,0.250,0.0,-0.0,16.221,-0.0,16.221
73,3.50,1.933,3.60,1,1,0.000,1.000,0.000,0.0,0.250,0.0,-0.0,23.315,-0.0,23.315
